In [9]:
pd.read_parquet('/home/cenciso/Documents/WORK/ENGIE/DashBoard/windShortTermForecast/dataset/currentGen.parquet')

,date,NombreEmpresa,Tipoinfoabrev,power,name
0,2025-09-19 00:30:00,ENGIE ENERGIA PERU S.A.A.,MW,121.3275,W.F. Punta Lomitas
1,2025-09-19 01:00:00,ENGIE ENERGIA PERU S.A.A.,MW,121.3275,W.F. Punta Lomitas
2,2025-09-19 01:30:00,ENGIE ENERGIA PERU S.A.A.,MW,114.4985,W.F. Punta Lomitas
3,2025-09-19 02:00:00,ENGIE ENERGIA PERU S.A.A.,MW,114.4985,W.F. Punta Lomitas
4,2025-09-19 02:30:00,ENGIE ENERGIA PERU S.A.A.,MW,107.2130,W.F. Punta Lomitas
...,...,...,...,...,...
2875,2025-09-24 22:00:00,ORYGEN PERU S.A.A.,MW,56.6980,W.F. Wayra I
2876,2025-09-24 22:30:00,ORYGEN PERU S.A.A.,MW,41.5760,W.F. Wayra I
2877,2025-09-24 23:00:00,ORYGEN PERU S.A.A.,MW,41.5760,W.F. Wayra I
2878,2025-09-24 23:30:00,ORYGEN PERU S.A.A.,MW,27.9155,W.F. Wayra I


In [41]:
# -*- coding:utf-8 -*-
#-----------------------------------------------------
# @Project: windShortTermForecast
# @File:    app.py
# @Author:  Carlos Enciso Ojeda
#-----------------------------------------------------
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc

# =======================
# Config de pesos y grupos
# =======================
WEIGHTS_FARMS = {
    "W.F. Cupisnique": 83.15, "W.F. Duna": 18.37, "W.F. Huambos": 18.37, "W.F. Marcona": 32.0,
    "W.F. Punta Lomitas": 296.4, "W.F. San Juan": 135.7, "W.F. Talara": 30.86,
    "W.F. Tres Hermanas": 97.15, "W.F. Wayra Ext": 177.0, "W.F. Wayra I": 132.30
}
GROUPS = {
    "Costa Norte": ["W.F. Cupisnique", "W.F. Talara"],
    "Costa Sur": ["W.F. Punta Lomitas", "W.F. Marcona", "W.F. Tres Hermanas", "W.F. Wayra Ext", "W.F. Wayra I"],
    "Sierra Centro": ["W.F. Huambos", "W.F. Duna", "W.F. San Juan"],
}

# =======================
# Load Generation Dataset
# =======================
PARQUET_CANDIDATES = [
    os.environ.get("DATASET_PATH"),
    "../dataset/currentGen.parquet",
    "./dataset/currentGen.parquet",
    "/opt/render/project/dataset/currentGen.parquet",
]
parquet_path = next((p for p in PARQUET_CANDIDATES if p and os.path.exists(p)), None)
if not parquet_path:
    raise FileNotFoundError("currentGen.parquet not found")

dataset = pd.read_parquet(parquet_path)
print(f"[OK] Data loaded from: {parquet_path} | shape={dataset.shape}")

# Validaciones mínimas
expected_cols = {"name", "date", "power"}
missing = expected_cols - set(dataset.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")
dataset = dataset.copy()
dataset["date"] = pd.to_datetime(dataset["date"])

# =======================
# Load Wind Forecast Dataset
# =======================
WIND_PARQUET_CANDIDATES = [
    os.environ.get("WIND_FCS_PATH"),
    "../dataset/windSpeedFcs.parquet",
    "./dataset/windSpeedFcs.parquet",
    "/opt/render/project/dataset/windSpeedFcs.parquet",
]
wind_parquet_path = next((p for p in WIND_PARQUET_CANDIDATES if p and os.path.exists(p)), None)

if wind_parquet_path:
    wind_dataset = pd.read_parquet(wind_parquet_path)
    print(f"[OK] Wind data loaded from: {wind_parquet_path} | shape={wind_dataset.shape}")
    # Mantener última corrida por modelo
    if "initDate" in wind_dataset.columns:
        wind_dataset["initDate"] = pd.to_datetime(wind_dataset["initDate"], errors="coerce")
        max_initDate_por_modelo = wind_dataset.groupby('model')['initDate'].transform('max')
        wind_dataset = wind_dataset[wind_dataset['initDate'] == max_initDate_por_modelo].copy()
    # Columnas mínimas
    wind_expected = {"name", "date", "wwind100", "model"}
    for col in wind_expected - set(wind_dataset.columns):
        wind_dataset[col] = pd.Series(dtype="float64" if col == "wwind100" else "object")
else:
    print("[WARN] windSpeedFcs.parquet not found; wind chart will be empty")
    wind_dataset = pd.DataFrame(columns=["name", "date", "wwind100", "model"])

wind_dataset = wind_dataset.copy()
wind_dataset["date"] = pd.to_datetime(wind_dataset["date"], errors="coerce")

# =======================
# App Configuration
# =======================
app = Dash(
    __name__,
    external_stylesheets=[
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"
    ],
)
app.title = "ENGIE WindForecast Platform"
server = app.server

# Theme settings
LIGHT_THEME = dbc.themes.CERULEAN
DARK_THEME  = dbc.themes.SUPERHERO

# =======================
# Styling Helpers
# =======================
def get_theme(dark: bool):
    colors = {
        "bg":   "#0f2537" if dark else "#ffffff",
        "text": "#ffffff" if dark else "#2c3e50",
    }
    template = "plotly_dark" if dark else "plotly_white"
    rs_style = dict(
        bgcolor="#000000" if dark else "#ffffff",
        activecolor="#4169e1" if dark else "#00bdff",
        font=dict(color=colors["text"]),
    )
    return colors, template, rs_style

def apply_common_layout(fig, *, dark: bool, ytitle: str, x_dtick_ms: int, step='backward'):
    colors, template, rs_style = get_theme(dark)
    fig.update_traces(line=dict(width=2))
    fig.update_layout(
        template=template,
        plot_bgcolor=colors["bg"],
        paper_bgcolor=colors["bg"],
        font=dict(color=colors["text"]),
        xaxis=dict(
            tickfont_size=10,
            showgrid=True, gridcolor="lightgrey", gridwidth=0.5,
            tickangle=0, tickformat="%d-%b\n%H:%M", dtick=x_dtick_ms, title=""
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey", gridwidth=0.5,
            title=dict(text=ytitle, font=dict(size=12))
        ),
        hovermode="x unified",
        margin=dict(t=5, b=20, l=10, r=10),
        autosize=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=.75),
        title=None,
        height=320
    )
    fig.update_xaxes(
        rangeslider_visible=False,
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1D", step="day", stepmode=step),
                dict(count=5, label="5D", step="day", stepmode=step),
                dict(step="all"),
            ],
            **rs_style,
        ),
    )
    return fig

# =======================
# UI Components
# =======================
def create_metric_card(icon_classes: str, title: str, value: str, change, color: str = "primary"):
    try:
        up = (str(change).startswith("+") or float(str(change).replace("+","").replace("%","")) >= 0)
    except Exception:
        up = True
    trend_class = "text-success" if up else "text-danger"
    change_txt  = f"{change}% from last month" if "%" not in str(change) else f"{change} from last month"
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([html.I(className=f"{icon_classes} fa-2x text-{color}")],
                         className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0", style={'font-size': '32px','font-weight': 'bold'}),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(change_txt, className=trend_class)
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")

metric_cards = [
    create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "primary"),
    create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "info"),
    create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
]

color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-sun", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-moon", html_for="switch"),
    ],
    className="my-2"
)

# Windfarm mapping
WINDFARM_MAPPING = {
    "0": "SEIN", "1": "W.F. Punta Lomitas", "2": "W.F. Cupisnique",
    "3": "W.F. Duna", "4": "W.F. Huambos", "5": "W.F. Marcona",
    "6": "W.F. San Juan", "7": "W.F. Talara", "8": "W.F. Tres Hermanas",
    "9": "W.F. Wayra Ext", "10": "W.F. Wayra I"
}

# =======================
# Helpers para series ponderadas por grupos
# =======================
def _weighted_series(df, names, value_col, weights):
    tmp = (df[df["name"].isin(names)]
           .assign(w=lambda x: x["name"].map(weights))
           .dropna(subset=["w"]))
    if tmp.empty:
        return pd.Series(dtype="float64")
    s = tmp.groupby("date").apply(lambda g: np.average(g[value_col], weights=g["w"]))
    s.index = pd.to_datetime(s.index)
    return s.sort_index()

def _build_group_figure(df, value_col, ytitle, dark, weights, x_dtick_ms):
    traces = []
    for gname, names in GROUPS.items():
        s = _weighted_series(df, names, value_col=value_col, weights=weights)
        traces.append(go.Scatter(x=s.index, y=s.values, mode="lines", name=gname))
    # Ponderado nacional (todas las granjas)
    all_names = list(weights.keys())
    s_all = _weighted_series(df, all_names, value_col=value_col, weights=weights)
    traces.append(go.Scatter(x=s_all.index, y=s_all.values, mode="lines",
                             name="SEIN (Total)", line=dict(width=3)))
    fig = go.Figure(data=traces)
    fig = apply_common_layout(fig, dark=bool(dark), ytitle=ytitle, x_dtick_ms=x_dtick_ms)
    return fig

# =======================
# App Layout
# =======================
app.layout = dbc.Container([
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),

    dbc.Row([
        # Left Panel
        dbc.Col([
            html.Div([
                html.Div(color_mode_switch, className='mb-3 text-end'),
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={'width': '200px','margin-bottom': '20px','display': 'block','margin-left': 'auto','margin-right': 'auto'}
                ),
                html.H1([html.Span("Welcome"), html.Br(), html.Span("WindForecast Platform")],
                        style={'color':'#00bdff'}, className="text-center mb-4"),
                html.P(
                    "This platform provides accurate wind forecasts and energy generation "
                    "insights for 10 wind farms and the SEIN system. It supports short and "
                    "long-term planning with interactive tools and clear visualizations.",
                    className="text-justify mb-4",
                ),
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value": 1},
                        {"label": "Long  Term (07 months)", "value": 2},
                    ],
                    value=1,
                ),
                html.Hr(),
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'}),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[{"label": v, "value": k} for k, v in WINDFARM_MAPPING.items()],
                    value="1",
                    className="mb-3"
                ),
            ])
        ], width=4),

        # Right Panel
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'}),
            dbc.Row([dbc.Col(card, width=4) for card in metric_cards]),
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='genPlot', style={'height': '350px'}),
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='windPlot', style={'height': '350px'}),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'}),
        ], width=8)
    ])
],
style={'display':'flex', 'maxWidth':'1440px'},
fluid=True)

# =======================
# Callbacks
# =======================
@app.callback(
    Output("theme-link", "href"),
    Output("theme-store", "data"),
    Input("switch", "value"),
    State("theme-store", "data"),
)
def toggle_theme(switch_value, data):
    dark = bool(switch_value)
    theme_href = DARK_THEME if dark else LIGHT_THEME
    return theme_href, {"dark": dark}

@app.callback(
    Output('genPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_gen_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    if windfarm_name == "SEIN":
        # 3 zonas + SEIN ponderado
        return _build_group_figure(dataset, value_col="power",
                                   ytitle="Wind Generation (MW)",
                                   dark=bool(dark), weights=WEIGHTS_FARMS,
                                   x_dtick_ms=18*3600*1000)

    subset = dataset.query("name == @windfarm_name").copy()
    if subset.empty:
        subset = pd.DataFrame({"date": pd.to_datetime([]), "power": []})

    fig = px.line(subset, x='date', y='power', line_shape='linear')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Generation (MW)',
                              x_dtick_ms=18*3600*1000)
    return fig

@app.callback(
    Output('windPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_wind_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    if windfarm_name == "SEIN":
        # Agregar por zonas ponderadas y promedio entre modelos
        def _weighted_series_wind(df, names, weights):
            tmp = (df[df["name"].isin(names)]
                   .assign(w=lambda x: x["name"].map(weights))
                   .dropna(subset=["w"]))
            if tmp.empty:
                return pd.Series(dtype="float64")
            # Pondero por granja dentro de cada modelo y fecha
            per_model = tmp.groupby(["date", "model"]).apply(
                lambda g: np.average(g["wwind100"], weights=g["w"])
            ).reset_index(name="v")
            # Promedio entre modelos por fecha
            s = per_model.groupby("date")["v"].mean()
            s.index = pd.to_datetime(s.index)
            return s.sort_index()

        traces = []
        for gname, names in GROUPS.items():
            s = _weighted_series_wind(wind_dataset, names, WEIGHTS_FARMS)
            traces.append(go.Scatter(x=s.index, y=s.values, mode="lines", name=gname))

        all_names = list(WEIGHTS_FARMS.keys())
        s_all = _weighted_series_wind(wind_dataset, all_names, WEIGHTS_FARMS)
        traces.append(go.Scatter(x=s_all.index, y=s_all.values, mode="lines",
                                 name="SEIN (Total)", line=dict(width=3)))

        fig = go.Figure(traces)
        fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Speed (m/s)',
                                  x_dtick_ms=24*3600*1000, step='todate')
        return fig

    # Caso normal: por parque, coloreado por modelo
    if not wind_dataset.empty and 'name' in wind_dataset.columns:
        subset = wind_dataset.query("name == @windfarm_name").copy()
    else:
        subset = pd.DataFrame(columns=["date", "wwind100", "model"])
        subset["date"] = pd.to_datetime(subset["date"])

    fig = px.line(subset, x='date', y='wwind100', color='model', line_shape='spline')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Speed (m/s)',
                              x_dtick_ms=24*3600*1000, step='todate')
    return fig

# =======================
# Main Execution
# =======================
if __name__ == '__main__':
    app.run(host="127.0.0.1", port=8050, debug=False)


[OK] Data loaded from: ../dataset/currentGen.parquet | shape=(2880, 5)
[OK] Wind data loaded from: ../dataset/windSpeedFcs.parquet | shape=(71270, 9)


In [42]:
# -*- coding:utf-8 -*-
#-----------------------------------------------------
# @Project: windShortTermForecast
# @File:    app.py
# @Author:  Carlos Enciso Ojeda
#-----------------------------------------------------
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc

# =======================
# Config de pesos y grupos
# =======================
WEIGHTS_FARMS = {
    "W.F. Cupisnique": 83.15, "W.F. Duna": 18.37, "W.F. Huambos": 18.37, "W.F. Marcona": 32.0,
    "W.F. Punta Lomitas": 296.4, "W.F. San Juan": 135.7, "W.F. Talara": 30.86,
    "W.F. Tres Hermanas": 97.15, "W.F. Wayra Ext": 177.0, "W.F. Wayra I": 132.30
}
GROUPS = {
    "Costa Norte": ["W.F. Cupisnique", "W.F. Talara"],
    "Costa Sur": ["W.F. Punta Lomitas", "W.F. Marcona", "W.F. Tres Hermanas", "W.F. Wayra Ext", "W.F. Wayra I"],
    "Sierra Centro": ["W.F. Huambos", "W.F. Duna", "W.F. San Juan"],
}

# =======================
# Load Generation Dataset
# =======================
PARQUET_CANDIDATES = [
    os.environ.get("DATASET_PATH"),
    "../dataset/currentGen.parquet",
    "./dataset/currentGen.parquet",
    "/opt/render/project/dataset/currentGen.parquet",
]
parquet_path = next((p for p in PARQUET_CANDIDATES if p and os.path.exists(p)), None)
if not parquet_path:
    raise FileNotFoundError("currentGen.parquet not found")

dataset = pd.read_parquet(parquet_path)
print(f"[OK] Data loaded from: {parquet_path} | shape={dataset.shape}")

# Validaciones mínimas
expected_cols = {"name", "date", "power"}
missing = expected_cols - set(dataset.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")
dataset = dataset.copy()
dataset["date"] = pd.to_datetime(dataset["date"])

# =======================
# Load Wind Forecast Dataset
# =======================
WIND_PARQUET_CANDIDATES = [
    os.environ.get("WIND_FCS_PATH"),
    "../dataset/windSpeedFcs.parquet",
    "./dataset/windSpeedFcs.parquet",
    "/opt/render/project/dataset/windSpeedFcs.parquet",
]
wind_parquet_path = next((p for p in WIND_PARQUET_CANDIDATES if p and os.path.exists(p)), None)

if wind_parquet_path:
    wind_dataset = pd.read_parquet(wind_parquet_path)
    print(f"[OK] Wind data loaded from: {wind_parquet_path} | shape={wind_dataset.shape}")
    # Mantener última corrida por modelo
    if "initDate" in wind_dataset.columns:
        wind_dataset["initDate"] = pd.to_datetime(wind_dataset["initDate"], errors="coerce")
        max_initDate_por_modelo = wind_dataset.groupby('model')['initDate'].transform('max')
        wind_dataset = wind_dataset[wind_dataset['initDate'] == max_initDate_por_modelo].copy()
    # Columnas mínimas
    wind_expected = {"name", "date", "wwind100", "model"}
    for col in wind_expected - set(wind_dataset.columns):
        wind_dataset[col] = pd.Series(dtype="float64" if col == "wwind100" else "object")
else:
    print("[WARN] windSpeedFcs.parquet not found; wind chart will be empty")
    wind_dataset = pd.DataFrame(columns=["name", "date", "wwind100", "model"])

wind_dataset = wind_dataset.copy()
wind_dataset["date"] = pd.to_datetime(wind_dataset["date"], errors="coerce")

# =======================
# App Configuration
# =======================
app = Dash(
    __name__,
    external_stylesheets=[
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"
    ],
)
app.title = "ENGIE WindForecast Platform"
server = app.server

# ==== NUEVAS PALETAS & TEMAS (experto en color) ====
# Bootstrap themes
LIGHT_THEME = dbc.themes.FLATLY   # limpio y moderno
DARK_THEME  = dbc.themes.DARKLY   # alto contraste, oscuro

# Paletas coherentes para todo Plotly
PALETTE_LIGHT = [
    "#2563eb",  # Azul (Costa Norte)
    "#ef4444",  # Rojo coral (Costa Sur)
    "#10b981",  # Verde (Sierra Centro)
    "#a855f7",  # Púrpura (SEIN)
    "#f59e0b",  # Ámbar
    "#06b6d4",  # Cian
    "#475569",  # Slate
]
PALETTE_DARK = [
    "#60a5fa",  # Azul claro
    "#f87171",  # Rojo claro
    "#34d399",  # Verde claro
    "#c084fc",  # Púrpura claro
    "#fbbf24",  # Ámbar
    "#22d3ee",  # Cian
    "#cbd5e1",  # Slate claro
]

def get_theme(dark: bool):
    """Colores base y colorway por tema"""
    colors = {
        "bg":   "#0b1220" if dark else "#f8fafc",   # fondo
        "text": "#e5e7eb" if dark else "#0f172a",   # texto
        "grid": "#334155" if dark else "#e2e8f0",   # grillas
        "accent": "#38bdf8" if dark else "#00bdff", # acento ENGIE
    }
    template = "plotly_dark" if dark else "plotly_white"
    colorway = PALETTE_DARK if dark else PALETTE_LIGHT
    rs_style = dict(
        bgcolor=colors["bg"],
        activecolor=colors["accent"],
        font=dict(color=colors["text"]),
    )
    return colors, template, rs_style, colorway

# =======================
# Styling Helpers
# =======================
def apply_common_layout(fig, *, dark: bool, ytitle: str, x_dtick_ms: int, step='backward'):
    colors, template, rs_style, colorway = get_theme(dark)
    fig.update_traces(line=dict(width=2))
    fig.update_layout(
        template=template,
        colorway=colorway,
        plot_bgcolor=colors["bg"],
        paper_bgcolor=colors["bg"],
        font=dict(color=colors["text"]),
        xaxis=dict(
            tickfont_size=10,
            showgrid=True, gridcolor=colors["grid"], gridwidth=0.6,
            tickangle=0, tickformat="%d-%b\n%H:%M", dtick=x_dtick_ms, title=""
        ),
        yaxis=dict(
            showgrid=True, gridcolor=colors["grid"], gridwidth=0.6,
            title=dict(text=ytitle, font=dict(size=12))
        ),
        hovermode="x unified",
        margin=dict(t=5, b=20, l=10, r=10),
        autosize=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=.75),
        title=None,
        height=320
    )
    fig.update_xaxes(
        rangeslider_visible=False,
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1D", step="day", stepmode=step),
                dict(count=5, label="5D", step="day", stepmode=step),
                dict(step="all", label="all"),
            ],
            **rs_style,
        ),
    )
    return fig

# =======================
# UI Components
# =======================
def create_metric_card(icon_classes: str, title: str, value: str, change, color: str = "primary"):
    try:
        up = (str(change).startswith("+") or float(str(change).replace("+","").replace("%","")) >= 0)
    except Exception:
        up = True
    trend_class = "text-success" if up else "text-danger"
    change_txt  = f"{change}% from last month" if "%" not in str(change) else f"{change} from last month"
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([html.I(className=f"{icon_classes} fa-2x text-{color}")],
                         className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0", style={'font-size': '32px','font-weight': 'bold'}),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(change_txt, className=trend_class)
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")

metric_cards = [
    create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "info"),
    create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "primary"),
    create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
]

color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-sun", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-moon", html_for="switch"),
    ],
    className="my-2"
)

# Windfarm mapping
WINDFARM_MAPPING = {
    "0": "SEIN", "1": "W.F. Punta Lomitas", "2": "W.F. Cupisnique",
    "3": "W.F. Duna", "4": "W.F. Huambos", "5": "W.F. Marcona",
    "6": "W.F. San Juan", "7": "W.F. Talara", "8": "W.F. Tres Hermanas",
    "9": "W.F. Wayra Ext", "10": "W.F. Wayra I"
}

# =======================
# Helpers para series ponderadas por grupos
# =======================
def _weighted_series(df, names, value_col, weights):
    tmp = (df[df["name"].isin(names)]
           .assign(w=lambda x: x["name"].map(weights))
           .dropna(subset=["w"]))
    if tmp.empty:
        return pd.Series(dtype="float64")
    s = tmp.groupby("date").apply(lambda g: np.average(g[value_col], weights=g["w"]))
    s.index = pd.to_datetime(s.index)
    return s.sort_index()

def _build_group_figure(df, value_col, ytitle, dark, weights, x_dtick_ms):
    traces = []
    # Respeta la asignación cromática: 0 Azul, 1 Rojo, 2 Verde, 3 Púrpura
    for gname, names in GROUPS.items():
        s = _weighted_series(df, names, value_col=value_col, weights=weights)
        traces.append(go.Scatter(x=s.index, y=s.values, mode="lines", name=gname))
    all_names = list(weights.keys())
    s_all = _weighted_series(df, all_names, value_col=value_col, weights=weights)
    traces.append(go.Scatter(x=s_all.index, y=s_all.values, mode="lines",
                             name="SEIN (Total)", line=dict(width=3)))
    fig = go.Figure(data=traces)
    fig = apply_common_layout(fig, dark=bool(dark), ytitle=ytitle, x_dtick_ms=x_dtick_ms)
    return fig

# =======================
# App Layout
# =======================
app.layout = dbc.Container([
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),

    dbc.Row([
        # Left Panel
        dbc.Col([
            html.Div([
                html.Div(color_mode_switch, className='mb-3 text-end'),
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={'width': '200px','margin-bottom': '20px','display': 'block','margin-left': 'auto','margin-right': 'auto'}
                ),
                html.H1([html.Span("Welcome"), html.Br(), html.Span("WindForecast Platform")],
                        style={'color':'#00bdff'}, className="text-center mb-4"),
                html.P(
                    "This platform provides accurate wind forecasts and energy generation "
                    "insights for 10 wind farms and the SEIN system. It supports short and "
                    "long-term planning with interactive tools and clear visualizations.",
                    className="text-justify mb-4",
                ),
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value": 1},
                        {"label": "Long  Term (07 months)", "value": 2},
                    ],
                    value=1,
                ),
                html.Hr(),
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'}),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[{"label": v, "value": k} for k, v in WINDFARM_MAPPING.items()],
                    value="1",
                    className="mb-3"
                ),
            ])
        ], width=4),

        # Right Panel
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'}),
            dbc.Row([dbc.Col(card, width=4) for card in metric_cards]),
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='genPlot', style={'height': '350px'}),
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='windPlot', style={'height': '350px'}),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'}),
        ], width=8)
    ])
],
style={'display':'flex', 'maxWidth':'1440px'},
fluid=True)

# =======================
# Callbacks
# =======================
@app.callback(
    Output("theme-link", "href"),
    Output("theme-store", "data"),
    Input("switch", "value"),
    State("theme-store", "data"),
)
def toggle_theme(switch_value, data):
    dark = bool(switch_value)
    theme_href = DARK_THEME if dark else LIGHT_THEME
    return theme_href, {"dark": dark}

@app.callback(
    Output('genPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_gen_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    if windfarm_name == "SEIN":
        # 3 zonas + SEIN ponderado
        return _build_group_figure(dataset, value_col="power",
                                   ytitle="Wind Generation (MW)",
                                   dark=bool(dark), weights=WEIGHTS_FARMS,
                                   x_dtick_ms=18*3600*1000)

    subset = dataset.query("name == @windfarm_name").copy()
    if subset.empty:
        subset = pd.DataFrame({"date": pd.to_datetime([]), "power": []})

    fig = px.line(subset, x='date', y='power', line_shape='linear')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Generation (MW)',
                              x_dtick_ms=18*3600*1000)
    return fig

@app.callback(
    Output('windPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_wind_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    if windfarm_name == "SEIN":
        # Agregar por zonas ponderadas y promedio entre modelos
        def _weighted_series_wind(df, names, weights):
            tmp = (df[df["name"].isin(names)]
                   .assign(w=lambda x: x["name"].map(weights))
                   .dropna(subset=["w"]))
            if tmp.empty:
                return pd.Series(dtype="float64")
            # Pondero por granja dentro de cada modelo y fecha
            per_model = tmp.groupby(["date", "model"]).apply(
                lambda g: np.average(g["wwind100"], weights=g["w"])
            ).reset_index(name="v")
            # Promedio entre modelos por fecha
            s = per_model.groupby("date")["v"].mean()
            s.index = pd.to_datetime(s.index)
            return s.sort_index()

        traces = []
        for gname, names in GROUPS.items():
            s = _weighted_series_wind(wind_dataset, names, WEIGHTS_FARMS)
            traces.append(go.Scatter(x=s.index, y=s.values, mode="lines", name=gname))

        all_names = list(WEIGHTS_FARMS.keys())
        s_all = _weighted_series_wind(wind_dataset, all_names, WEIGHTS_FARMS)
        traces.append(go.Scatter(x=s_all.index, y=s_all.values, mode="lines",
                                 name="SEIN (Total)", line=dict(width=3)))

        fig = go.Figure(traces)
        fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Speed (m/s)',
                                  x_dtick_ms=24*3600*1000, step='todate')
        return fig

    # Caso normal: por parque, coloreado por modelo
    if not wind_dataset.empty and 'name' in wind_dataset.columns:
        subset = wind_dataset.query("name == @windfarm_name").copy()
    else:
        subset = pd.DataFrame(columns=["date", "wwind100", "model"])
        subset["date"] = pd.to_datetime(subset["date"])

    fig = px.line(subset, x='date', y='wwind100', color='model', line_shape='spline')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Speed (m/s)',
                              x_dtick_ms=24*3600*1000, step='todate')
    return fig

# =======================
# Main Execution
# =======================
if __name__ == '__main__':
    app.run(host="127.0.0.1", port=8050, debug=False)


[OK] Data loaded from: ../dataset/currentGen.parquet | shape=(2880, 5)
[OK] Wind data loaded from: ../dataset/windSpeedFcs.parquet | shape=(71270, 9)


In [46]:
30.86+83.15

114.01

In [51]:
total = np.array([x for x in WEIGHTS_FARMS.values()]).sum()
frac = 100*(296.4+32.0+135.7+97.15+177+132.30)/total
frac

np.float64(85.2394007637325)

In [53]:
# -*- coding:utf-8 -*-
#-----------------------------------------------------
# @Project: windShortTermForecast
# @File:    app.py
# @Author:  Carlos Enciso Ojeda
#-----------------------------------------------------
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc

# =======================
# Config de pesos y grupos
# =======================
WEIGHTS_FARMS = {
    "W.F. Cupisnique": 83.15, "W.F. Duna": 18.37, "W.F. Huambos": 18.37, "W.F. Marcona": 32.0,
    "W.F. Punta Lomitas": 296.4, "W.F. San Juan": 135.7, "W.F. Talara": 30.86,
    "W.F. Tres Hermanas": 97.15, "W.F. Wayra Ext": 177.0, "W.F. Wayra I": 132.30
}
GROUPS = {
    "Costa Norte (11%)": ["W.F. Cupisnique", "W.F. Talara"],
    "Costa Sur (85%)": ["W.F. Punta Lomitas", "W.F. Marcona", "W.F. San Juan", "W.F. Tres Hermanas", "W.F. Wayra Ext", "W.F. Wayra I"],
    "Sierra Norte (4%)": ["W.F. Huambos", "W.F. Duna"],
}

# =======================
# Load Generation Dataset
# =======================
PARQUET_CANDIDATES = [
    os.environ.get("DATASET_PATH"),
    "../dataset/currentGen.parquet",
    "./dataset/currentGen.parquet",
    "/opt/render/project/dataset/currentGen.parquet",
]
parquet_path = next((p for p in PARQUET_CANDIDATES if p and os.path.exists(p)), None)
if not parquet_path:
    raise FileNotFoundError("currentGen.parquet not found")

dataset = pd.read_parquet(parquet_path)
print(f"[OK] Data loaded from: {parquet_path} | shape={dataset.shape}")

# Validaciones mínimas
expected_cols = {"name", "date", "power"}
missing = expected_cols - set(dataset.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")
dataset = dataset.copy()
dataset["date"] = pd.to_datetime(dataset["date"])

# =======================
# Load Wind Forecast Dataset
# =======================
WIND_PARQUET_CANDIDATES = [
    os.environ.get("WIND_FCS_PATH"),
    "../dataset/windSpeedFcs.parquet",
    "./dataset/windSpeedFcs.parquet",
    "/opt/render/project/dataset/windSpeedFcs.parquet",
]
wind_parquet_path = next((p for p in WIND_PARQUET_CANDIDATES if p and os.path.exists(p)), None)

if wind_parquet_path:
    wind_dataset = pd.read_parquet(wind_parquet_path)
    print(f"[OK] Wind data loaded from: {wind_parquet_path} | shape={wind_dataset.shape}")
    if "initDate" in wind_dataset.columns:
        wind_dataset["initDate"] = pd.to_datetime(wind_dataset["initDate"], errors="coerce")
        max_initDate_por_modelo = wind_dataset.groupby('model')['initDate'].transform('max')
        wind_dataset = wind_dataset[wind_dataset['initDate'] == max_initDate_por_modelo].copy()
    wind_expected = {"name", "date", "wwind100", "model"}
    for col in wind_expected - set(wind_dataset.columns):
        wind_dataset[col] = pd.Series(dtype="float64" if col == "wwind100" else "object")
else:
    print("[WARN] windSpeedFcs.parquet not found; wind chart will be empty")
    wind_dataset = pd.DataFrame(columns=["name", "date", "wwind100", "model"])

wind_dataset = wind_dataset.copy()
wind_dataset["date"] = pd.to_datetime(wind_dataset["date"], errors="coerce")

# =======================
# App Configuration
# =======================
app = Dash(
    __name__,
    external_stylesheets=[
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"
    ],
)
app.title = "ENGIE WindForecast Platform"
server = app.server

# === Mantén tus temas Bootstrap favoritos ===
LIGHT_THEME = dbc.themes.CERULEAN
DARK_THEME  = dbc.themes.SUPERHERO

# === SOLO paletas para las líneas (armonizadas con cada tema) ===
LIGHT_COLORWAY = [
    "#0d6efd",  # Azul (Costa Norte)
    "#e83e8c",  # Rosa/coral (Costa Sur)
    "#20c997",  # Verde-azulado (Sierra Centro)
    "#6f42c1",  # Púrpura (SEIN Total)
    "#fd7e14",  # Ámbar
    "#198754",  # Verde
    "#495057",  # Slate
]
DARK_COLORWAY = [
    "#7ab8ff",  # Azul claro
    "#ff8fa3",  # Rosa claro
    "#34d0ba",  # Teal claro
    "#c29ffa",  # Púrpura claro
    "#f8b26a",  # Ámbar
    "#8ce99a",  # Verde
    "#cbd5e1",  # Slate claro
]

def get_theme(dark: bool):
    colors = {
        "bg":   "#0f2537" if dark else "#ffffff",
        "text": "#ffffff" if dark else "#2c3e50",
        "grid": "#90a4b4" if dark else "#e5e7eb",
        "accent": "#00bdff" if not dark else "#8dd3ff",
    }
    template = "plotly_dark" if dark else "plotly_white"
    colorway = DARK_COLORWAY if dark else LIGHT_COLORWAY
    rs_style = dict(
        bgcolor=colors["bg"],
        activecolor=colors["accent"],
        font=dict(color=colors["text"]),
    )
    return colors, template, rs_style, colorway

# =======================
# Styling Helpers
# =======================
def apply_common_layout(fig, *, dark: bool, ytitle: str, x_dtick_ms: int, step='backward'):
    colors, template, rs_style, colorway = get_theme(dark)
    fig.update_traces(line=dict(width=2))
    fig.update_layout(
        template=template,
        colorway=colorway,             # <<<<<< clave: líneas armonizadas
        plot_bgcolor=colors["bg"],
        paper_bgcolor=colors["bg"],
        font=dict(color=colors["text"]),
        xaxis=dict(
            tickfont_size=10,
            showgrid=True, gridcolor=colors["grid"], gridwidth=0.6,
            tickangle=0, tickformat="%d-%b\n%H:%M", dtick=x_dtick_ms, title=""
        ),
        yaxis=dict(
            showgrid=True, gridcolor=colors["grid"], gridwidth=0.6,
            title=dict(text=ytitle, font=dict(size=12))
        ),
        hovermode="x unified",
        margin=dict(t=5, b=50, l=10, r=10),
        autosize=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=.75),
        title=None,
        height=320
    )
    fig.update_xaxes(
        rangeslider_visible=False,
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1D", step="day", stepmode=step),
                dict(count=5, label="5D", step="day", stepmode=step),
                dict(step="all", label="all"),
            ],
            **rs_style,
        ),
    )
    return fig

# =======================
# UI Components
# =======================
def create_metric_card(icon_classes: str, title: str, value: str, change, color: str = "primary"):
    try:
        up = (str(change).startswith("+") or float(str(change).replace("+","").replace("%","")) >= 0)
    except Exception:
        up = True
    trend_class = "text-success" if up else "text-danger"
    change_txt  = f"{change}% from last month" if "%" not in str(change) else f"{change} from last month"
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([html.I(className=f"{icon_classes} fa-2x text-{color}")],
                         className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0", style={'font-size': '32px','font-weight': 'bold'}),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(change_txt, className=trend_class)
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")

metric_cards = [
    create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "primary"),
    create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "info"),
    create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
]

color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-sun", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-moon", html_for="switch"),
    ],
    className="my-2"
)

# Windfarm mapping
WINDFARM_MAPPING = {
    "0": "SEIN", "1": "W.F. Punta Lomitas", "2": "W.F. Cupisnique",
    "3": "W.F. Duna", "4": "W.F. Huambos", "5": "W.F. Marcona",
    "6": "W.F. San Juan", "7": "W.F. Talara", "8": "W.F. Tres Hermanas",
    "9": "W.F. Wayra Ext", "10": "W.F. Wayra I"
}

# =======================
# Helpers para series ponderadas por grupos
# =======================
def _weighted_series(df, names, value_col, weights):
    tmp = (df[df["name"].isin(names)]
           .assign(w=lambda x: x["name"].map(weights))
           .dropna(subset=["w"]))
    if tmp.empty:
        return pd.Series(dtype="float64")
    s = tmp.groupby("date").apply(lambda g: np.average(g[value_col], weights=g["w"]))
    s.index = pd.to_datetime(s.index)
    return s.sort_index()

def _build_group_figure(df, value_col, ytitle, dark, weights, x_dtick_ms):
    traces = []
    for gname, names in GROUPS.items():
        s = _weighted_series(df, names, value_col=value_col, weights=weights)
        traces.append(go.Scatter(x=s.index, y=s.values, mode="lines", name=gname))
    all_names = list(weights.keys())
    s_all = _weighted_series(df, all_names, value_col=value_col, weights=weights)
    traces.append(go.Scatter(x=s_all.index, y=s_all.values, mode="lines",
                             name="SEIN (Total)", line=dict(width=3)))
    fig = go.Figure(data=traces)
    fig = apply_common_layout(fig, dark=bool(dark), ytitle=ytitle, x_dtick_ms=x_dtick_ms)
    return fig

# =======================
# App Layout
# =======================
app.layout = dbc.Container([
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),

    dbc.Row([
        # Left Panel
        dbc.Col([
            html.Div([
                html.Div(color_mode_switch, className='mb-3 text-end'),
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={'width': '200px','margin-bottom': '20px','display': 'block','margin-left': 'auto','margin-right': 'auto'}
                ),
                html.H1([html.Span("Welcome"), html.Br(), html.Span("WindForecast Platform")],
                        style={'color':'#00bdff'}, className="text-center mb-4"),
                html.P(
                    "This platform provides accurate wind forecasts and energy generation "
                    "insights for 10 wind farms and the SEIN system. It supports short and "
                    "long-term planning with interactive tools and clear visualizations.",
                    className="text-justify mb-4",
                ),
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value": 1},
                        {"label": "Long  Term (07 months)", "value": 2},
                    ],
                    value=1,
                ),
                html.Hr(),
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'}),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[{"label": v, "value": k} for k, v in WINDFARM_MAPPING.items()],
                    value="1",
                    className="mb-3"
                ),
            ])
        ], width=4),

        # Right Panel
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'}),
            dbc.Row([dbc.Col(card, width=4) for card in metric_cards]),
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='genPlot', style={'height': '350px'}),
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='windPlot', style={'height': '350px'}),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'}),
        ], width=8)
    ])
],
style={'display':'flex', 'maxWidth':'1440px'},
fluid=True)

# =======================
# Callbacks
# =======================
@app.callback(
    Output("theme-link", "href"),
    Output("theme-store", "data"),
    Input("switch", "value"),
    State("theme-store", "data"),
)
def toggle_theme(switch_value, data):
    dark = bool(switch_value)
    theme_href = DARK_THEME if dark else LIGHT_THEME
    return theme_href, {"dark": dark}

@app.callback(
    Output('genPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_gen_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    if windfarm_name == "SEIN":
        return _build_group_figure(dataset, value_col="power",
                                   ytitle="Wind Generation (MW)",
                                   dark=bool(dark), weights=WEIGHTS_FARMS,
                                   x_dtick_ms=18*3600*1000)

    subset = dataset.query("name == @windfarm_name").copy()
    if subset.empty:
        subset = pd.DataFrame({"date": pd.to_datetime([]), "power": []})

    fig = px.line(subset, x='date', y='power', line_shape='linear')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Generation (MW)',
                              x_dtick_ms=18*3600*1000)
    return fig

@app.callback(
    Output('windPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_wind_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    if windfarm_name == "SEIN":
        def _weighted_series_wind(df, names, weights):
            tmp = (df[df["name"].isin(names)]
                   .assign(w=lambda x: x["name"].map(weights))
                   .dropna(subset=["w"]))
            if tmp.empty:
                return pd.Series(dtype="float64")
            per_model = tmp.groupby(["date", "model"]).apply(
                lambda g: np.average(g["wwind100"], weights=g["w"])
            ).reset_index(name="v")
            s = per_model.groupby("date")["v"].mean()
            s.index = pd.to_datetime(s.index)
            return s.sort_index()

        traces = []
        for gname, names in GROUPS.items():
            s = _weighted_series_wind(wind_dataset, names, WEIGHTS_FARMS)
            traces.append(go.Scatter(x=s.index, y=s.values, mode="lines", name=gname))

        all_names = list(WEIGHTS_FARMS.keys())
        s_all = _weighted_series_wind(wind_dataset, all_names, WEIGHTS_FARMS)
        traces.append(go.Scatter(x=s_all.index, y=s_all.values, mode="lines",
                                 name="SEIN (Total)", line=dict(width=3)))

        fig = go.Figure(traces)
        fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Speed (m/s)',
                                  x_dtick_ms=24*3600*1000, step='todate')
        return fig

    if not wind_dataset.empty and 'name' in wind_dataset.columns:
        subset = wind_dataset.query("name == @windfarm_name").copy()
    else:
        subset = pd.DataFrame(columns=["date", "wwind100", "model"])
        subset["date"] = pd.to_datetime(subset["date"])

    fig = px.line(subset, x='date', y='wwind100', color='model', line_shape='spline')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Speed (m/s)',
                              x_dtick_ms=24*3600*1000, step='todate')
    return fig

# =======================
# Main Execution
# =======================
if __name__ == '__main__':
    app.run(host="127.0.0.1", port=8050, debug=False)


[OK] Data loaded from: ../dataset/currentGen.parquet | shape=(2880, 5)
[OK] Wind data loaded from: ../dataset/windSpeedFcs.parquet | shape=(71270, 9)


In [18]:
# -*- coding:utf-8 -*-
#-----------------------------------------------------
# @Project: ./windShortTermForecast/app/
# @File:    ./windShortTermForecast/app/app.py
# @Author:  Carlos Enciso Ojeda
#-----------------------------------------------------
# -- Imports -- #
import os
import dash
import pandas as pd
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px

# =======================
# Read Gen Dataset (Parquet)
# =======================
# Prioridad de rutas: ENV -> local repo -> Render path
PARQUET_CANDIDATES = [
    os.environ.get("DATASET_PATH"),
    "../dataset/currentGen.parquet",
    "./dataset/currentGen.parquet",
    "/opt/render/project/dataset/currentGen.parquet",
]
parquet_path = next((p for p in PARQUET_CANDIDATES if p and os.path.exists(p)), None)

if parquet_path:
    dataset = pd.read_parquet(parquet_path)
    print(f"[OK] Data loaded from: {parquet_path} | shape={dataset.shape}")
else:
    raise FileNotFoundError(
        "No se encontró 'currentGen.parquet'. "
        "Colócalo en ./dataset/ o ../dataset/ o define DATASET_PATH."
    )

# Validación mínima de columnas esperadas
expected_cols = {"name", "date", "power"}
missing = expected_cols - set(dataset.columns)
if missing:
    raise ValueError(f"El Parquet no contiene columnas requeridas {expected_cols}. Faltan: {missing}")

# Asegurar tipos
dataset = dataset.copy()
dataset["date"] = pd.to_datetime(dataset["date"])

# =======================
# Read Wind Forecast Dataset (Parquet)
# =======================
WIND_PARQUET_CANDIDATES = [
    os.environ.get("WIND_FCS_PATH"),
    "../dataset/windSpeedFcs.parquet",
    "./dataset/windSpeedFcs.parquet",
    "/opt/render/project/dataset/windSpeedFcs.parquet",
]
wind_parquet_path = next((p for p in WIND_PARQUET_CANDIDATES if p and os.path.exists(p)), None)

if wind_parquet_path:
    wind_dataset = pd.read_parquet(wind_parquet_path)
    print(f"[OK] Wind data loaded from: {wind_parquet_path} | shape={wind_dataset.shape}")
    wind_expected = {"name", "date", "wwind100", "model"}
    wmissing = wind_expected - set(wind_dataset.columns)
    if wmissing:
        for col in wmissing:
            wind_dataset[col] = pd.Series(dtype="float64" if col == "wwind100" else "object")
    wind_dataset = wind_dataset.copy()
    wind_dataset["date"] = pd.to_datetime(wind_dataset["date"], errors="coerce")
else:
    print("[WARN] No se encontró 'windSpeedFcs.parquet'; el gráfico de viento se mostrará vacío.")
    wind_dataset = pd.DataFrame(columns=["name", "date", "wwind100", "model"])
    wind_dataset["date"] = pd.to_datetime(wind_dataset["date"])

# =======================
# App & Theme Setup
# =======================
app = Dash(
    __name__,
    external_stylesheets=[
        # No cargar tema dbc aquí; lo alternamos con <link> dinámico
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"
    ],
)
app.title = "ENGIE WindForecast Platform"
server = app.server  # por si luego lo usas en un WSGI

# Temas (light/dark)
LIGHT_THEME = dbc.themes.CERULEAN
DARK_THEME  = dbc.themes.SUPERHERO

# =======================
# Helpers de estilo (unifica el look & feel de ambas figuras)
# =======================
def get_theme(dark: bool):
    colors = {
        "bg":   "#0f2537" if dark else "#ffffff",
        "text": "#ffffff" if dark else "#2c3e50",
    }
    template = "plotly_dark" if dark else "plotly_white"
    rs_style = dict(
        bgcolor="#000000" if dark else "#ffffff",
        activecolor="#4169e1" if dark else "#00bdff",
        font=dict(color=colors["text"]),
    )
    return colors, template, rs_style

def apply_common_layout(fig, *, dark: bool, ytitle: str, x_dtick_ms: int):
    colors, template, rs_style = get_theme(dark)
    fig.update_traces(line=dict(width=2))
    fig.update_layout(
        template=template,
        plot_bgcolor=colors["bg"],
        paper_bgcolor=colors["bg"],
        font=dict(color=colors["text"]),
        xaxis=dict(
            tickfont_size=10,
            showgrid=True, gridcolor="lightgrey", gridwidth=0.01, griddash="solid",
            tickangle=0, tickformat="%d-%b\n%H:%M", dtick=x_dtick_ms, title=""
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey", gridwidth=0.01, griddash="solid",
            title=dict(text=ytitle, font=dict(size=12))
        ),
        hovermode="x unified",
        margin=dict(t=60, b=5, l=10, r=10),
        autosize=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        title=None,
    )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1D", step="day", stepmode="backward"),
                dict(count=5, label="5D", step="day", stepmode="backward"),
                dict(step="all"),
            ],
            **rs_style,
        ),
    )
    return fig

# =======================
# UI Bits
# =======================
def create_metric_card(icon_classes: str, title: str, value: str, change, color: str = "primary"):
    try:
        up = (str(change).startswith("+") or float(str(change).replace("+","").replace("%","")) >= 0)
    except Exception:
        up = True
    trend_class = "text-success" if up else "text-danger"
    change_txt  = f"{change}% from last month" if "%" not in str(change) else f"{change} from last month"
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([html.I(className=f"{icon_classes} fa-2x text-{color}")],
                         className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0", style={'font-size': '32px','font-weight': 'bold'}),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(change_txt, className=trend_class)
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")

metric_cards = [
    create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "primary"),
    create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "info"),
    create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
]

color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-sun", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-moon", html_for="switch"),
    ],
    className="my-2"
)

# Mapeo de valores del RadioItem -> nombres en dataset['name']
WINDFARM_MAPPING = {
    "0": "SEIN", "1": "W.F. Punta Lomitas", "2": "W.F. Cupisnique",
    "3": "W.F. Duna", "4": "W.F. Huambos", "5": "W.F. Marcona",
    "6": "W.F. San Juan", "7": "W.F. Talara", "8": "W.F. Tres Hermanas",
    "9": "W.F. Wayra Ext", "10": "W.F. Wayra I"
}

# =======================
# Layout
# =======================
app.layout = dbc.Container([
    # Theme link y estado (para toggle dinámico)
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),

    dbc.Row([
        # -------- Left Panel -------- #
        dbc.Col([
            html.Div([
                html.Div(color_mode_switch, className='mb-3 text-end'),
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={'width': '200px','margin-bottom': '20px','display': 'block','margin-left': 'auto','margin-right': 'auto'}
                ),
                html.H1([html.Span("Welcome"), html.Br(), html.Span("WindForecast Platform")],
                        style={'color':'#00bdff'}, className="text-center mb-4"),
                html.P(
                    "This platform provides accurate wind forecasts and energy generation "
                    "insights for 10 wind farms and the SEIN system. It supports short and "
                    "long-term planning with interactive tools and clear visualizations.",
                    className="text-justify mb-4",
                ),
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value": 1},
                        {"label": "Long  Term (07 months)", "value": 2},
                    ],
                    value=1,
                ),
                html.Hr(),
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'}),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[{"label": v, "value": k} for k, v in WINDFARM_MAPPING.items()],
                    value="1",
                    className="mb-3"
                ),
            ])
        ], width=4),

        # -------- Right Panel -------- #
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'}),
            dbc.Row([dbc.Col(card, width=4) for card in metric_cards]),
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='genPlot', style={'height': '400px'}),
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='windPlot', style={'height':'250px'}),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'}),
        ], width=8)
    ])
],
style={'display':'flex', 'maxWidth':'1440px'},
fluid=True)

# =======================
# Callbacks
# =======================
@app.callback(
    Output("theme-link", "href"),
    Output("theme-store", "data"),
    Input("switch", "value"),
    State("theme-store", "data"),
)
def toggle_theme(switch_value, data):
    dark = bool(switch_value)
    theme_href = DARK_THEME if dark else LIGHT_THEME
    return theme_href, {"dark": dark}

@app.callback(
    Output('genPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_gen_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    subset = dataset.query("name == @windfarm_name").copy()
    if subset.empty:
        subset = pd.DataFrame({"date": pd.to_datetime([]), "power": []})

    fig = px.line(subset, x='date', y='power', line_shape='linear')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Generation (MW)',
                              x_dtick_ms=18*3600*1000)  # 18 horas
    return fig

@app.callback(
    Output('windPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def updateWindPlot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    if not wind_dataset.empty and 'name' in wind_dataset.columns:
        subset = wind_dataset.query("name == @windfarm_name").copy()
    else:
        subset = pd.DataFrame(columns=["date", "wwind100", "model"])
        subset["date"] = pd.to_datetime(subset["date"])

    fig = px.line(subset, x='date', y='wwind100', color='model', line_shape='spline')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Speed (m/s)',
                              x_dtick_ms=24*3600*1000)  # 24 horas
    return fig

# =======================
# Main
# =======================
if __name__ == '__main__':
    # Local run
    app.run(host="127.0.0.1", port=8050, debug=False)

[OK] Data loaded from: ../dataset/currentGen.parquet | shape=(2880, 5)
[OK] Wind data loaded from: ../dataset/windSpeedFcs.parquet | shape=(71270, 9)


In [19]:
# -*- coding:utf-8 -*-
#-----------------------------------------------------
# @Project: ./windShortTermForecast/app/
# @File: ./windShortTermForecast/app/app.py
# @Author: Carlos Enciso Ojeda
# @Email: carlos.enciso.o@gmail.com
#-----------------------------------------------------
#-- Import modules --#
import dash
import time
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

#-- Function to generate sample data for demonstration --#
def generate_sample_data(windfarm_id):
    """Generate sample wind generation data for different wind farms"""
    np.random.seed(int(windfarm_id))  # For consistent results
    
    # Create date range for the next 15 days
    dates = [datetime.now() + timedelta(hours=i) for i in range(24*15)]
    
    # Different generation patterns based on windfarm
    if windfarm_id == "0":  # SEIN
        base_power = np.random.uniform(800, 1200, len(dates))
        daily_pattern = np.sin(np.linspace(0, 4*np.pi, len(dates))) * 200
        noise = np.random.normal(0, 50, len(dates))
        generation = base_power + daily_pattern + noise
        farm_name = "SEIN System"
    else:
        # Individual wind farms have different patterns
        base_multiplier = int(windfarm_id) * 50
        base_power = np.random.uniform(100 + base_multiplier, 200 + base_multiplier, len(dates))
        daily_pattern = np.sin(np.linspace(0, 4*np.pi, len(dates))) * (50 + int(windfarm_id)*5)
        noise = np.random.normal(0, 10 + int(windfarm_id), len(dates))
        generation = base_power + daily_pattern + noise
        farm_name = f"W.F. {['Punta Lomitas', 'Cupisnique', 'Duna', 'Huambos', 'Marcona', 'San Juan', 'Talara', 'Tres Hermanas', 'Wayra Ext.', 'Wayra I'][int(windfarm_id)-1]}"
    
    return pd.DataFrame({
        'datetime': dates,
        'generation_mw': generation,
        'windfarm': farm_name
    })

#-- Layout --#
app = Dash(__name__,
           external_stylesheets=[
               dbc.themes.SUPERHERO,
               "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"],
           assets_folder='/content/assets',
           )
#-- Tab Name --#
app.title = "ENGIE WindForecast Platform "

#-- Theme to Toggle --#
LIGHT_THEME = dbc.themes.SUPERHERO
DARK_THEME = dbc.themes.SUPERHERO

#-- Cards --#
def create_metric_card(icon, title, value, change, color="primary"):
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([
                    html.I(className=f"fas {icon} fa-2x text-{color}")
                ], className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0",
                            style={
                                'font-size': '32px',
                                'font-weight': 'bold',
                                }),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(f"{change}% from last month",
                              className=f"text-{'success' if '+' in change else 'danger'}")
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")

metric_cards = [
    create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "primary"),
    create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "info"),
    create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
]

#-- UI Theme switch --#
color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-moon", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-sun", html_for="switch"),
    ],
    className="my-2"
)

#-- Main Layout --#
app.layout = dbc.Container([
    #-- Theme link and store --#
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),
    
    #-- Left Panel --#
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Div(color_mode_switch, className='mb-3 text-end'),
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={
                        'width': '200px',
                        'margin-bottom': '20px',
                        'display': 'block',
                        'margin-left': 'auto',
                        'margin-right': 'auto'
                    }
                ),
                html.H1([
                    html.Span("Welcome"),
                    html.Br(),
                    html.Span("WindForecast Platform")
                ], style={'color':'#00bdff'}, className="text-center mb-4"),
                html.P(
                    "This platform provides accurate wind forecasts and energy generation \
                    insights for 10 wind farms and the SEIN system. It supports short and \
                    long-term planning with interactive tools and clear visualizations.",
                    className="text-justify mb-4",
                ),
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value":1},
                        {"label": "Long  Term (07 months)", "value":2},
                    ],
                    value=1,
                ),
                html.Hr(),
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'}),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[
                        {"label": "SEIN", "value": "0"},
                        {"label": "W.F. Punta Lomitas", "value": "1"},
                        {"label": "W.F. Cupisnique", "value": "2"},
                        {"label": "W.F. Duna", "value": "3"},
                        {"label": "W.F. Huambos", "value": "4"},
                        {"label": "W.F. Marcona", "value": "5"},
                        {"label": "W.F. San Juan", "value": "6"},
                        {"label": "W.F. Talara", "value": "7"},
                        {"label": "W.F. Tres Hermanas", "value": "8"},
                        {"label": "W.F. Wayra Ext.", "value": "9"},
                        {"label": "W.F. Wayra I", "value": "10"},
                    ],
                    value="0",
                    className="mb-3"
                ),
            ])
        ], width=4),
        
        #-- Right Panel --#
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'}),
            dbc.Row([dbc.Col(card, width=4) for card in metric_cards]),
            
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'}),
            #-- Add the generation plot here --#
            dcc.Graph(id='generation-plot', style={'height': '400px'}),
            
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'}),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'}),
        ], width=8)
    ])
], fluid=True)

#-- Callback to update the generation plot based on windfarm selection --#
@app.callback(
    Output('generation-plot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input('top-buttons', 'value')
)
def update_generation_plot(windfarm_id, forecast_range):
    # Generate sample data based on selected windfarm
    df = generate_sample_data(windfarm_id)
    
    # Create the plot using Plotly Express
    fig = px.line(
        df, 
        x='datetime', 
        y='generation_mw',
        title=f'Wind Generation Forecast - {df["windfarm"].iloc[0]}',
        labels={'datetime': 'Date', 'generation_mw': 'Generation (MW)'}
    )
    
    # Customize the plot appearance
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font_color='white',
        xaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray'),
        yaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray'),
        hovermode='x unified',
        #-- Legend --#
        legend=dict(
        orientation='h',
        x=0.5, xanchor='center',
        y=-0.2, yanchor='top',     # move further down if still overlaps (e.g., -0.25)
        bgcolor='rgba(0,0,0,0)'
    ),
        margin=dict(t=60, r=20, b=80, l=20)
    )
    
    # Add a range slider for better navigation
    fig.update_xaxes(rangeslider_visible=True)
    
    return fig

if __name__ == '__main__':
    app.run(mode='inline', debug=False)

In [ ]:
CUPISNIQUE: 83.15
DUNA:18.37
HUAMBOS: 18.37
MARCONA: 32
PUNTA LOMITAS: 296.4 (PLOM: 260, Ext.: 36.4)
SAN JUAN: 135.7
TALARA: 30.86 
TRES HERMANAS: 97.15
WAYRA EXT.: 177
WAYRA: 132.30

In [23]:
sein = (
    wind_dataset
    .assign(weight=wind_dataset["name"].map(weights))
    .groupby(["initDate", "date", "model", "nleadHour", "nleadDays"], group_keys=False)
    .apply(lambda g: pd.Series({
        "wwind10": np.average(g["wwind10"], weights=g["weight"]),
        "t2m": np.average(g["t2m"], weights=g["weight"]),
        "wwind100": np.average(g["wwind100"], weights=g["weight"]),
        "name": "SEIN"
    }))
    .reset_index()
)

wind_dataset = pd.concat([wind_dataset, sein], ignore_index=True)

/tmp/ipykernel_17535/4278950361.py:5: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [25]:
wind_dataset

,initDate,date,name,wwind10,t2m,model,wwind100,nleadHour,nleadDays
0,2025-09-16 19:00:00,2025-09-16 19:00:00,W.F. Cupisnique,5.560980,17.248041,GFS,7.563892,0,0
1,2025-09-16 19:00:00,2025-09-16 20:00:00,W.F. Cupisnique,4.835906,16.950891,GFS,6.577665,1,0
2,2025-09-16 19:00:00,2025-09-16 21:00:00,W.F. Cupisnique,3.800976,16.741693,GFS,5.169982,2,0
3,2025-09-16 19:00:00,2025-09-16 22:00:00,W.F. Cupisnique,3.584989,16.497949,GFS,4.876203,3,0
4,2025-09-16 19:00:00,2025-09-16 23:00:00,W.F. Cupisnique,3.131374,16.363397,GFS,4.259208,4,0
...,...,...,...,...,...,...,...,...,...
92646,2025-09-23 19:00:00,2025-10-07 19:00:00,SEIN,4.217757,15.982382,IFS,5.728569,336,14
92647,2025-09-23 19:00:00,2025-10-08 01:00:00,SEIN,2.395027,14.141643,IFS,3.614253,342,14
92648,2025-09-23 19:00:00,2025-10-08 07:00:00,SEIN,2.835604,14.196021,IFS,4.186774,348,14
92649,2025-09-23 19:00:00,2025-10-08 13:00:00,SEIN,5.110139,20.988018,IFS,6.232429,354,14


In [24]:
weights = {
    "W.F. Cupisnique": 83.15, "W.F. Duna": 18.37, "W.F. Huambos": 18.37,
    "W.F. Marcona": 32, "W.F. Punta Lomitas": 296.4, "W.F. San Juan": 135.7,
    "W.F. Talara": 30.86, "W.F. Tres Hermanas": 97.15,
    "W.F. Wayra Ext": 177, "W.F. Wayra I": 132.30
}
sein = (wind_dataset.assign(weight=wind_dataset["name"].map(weights))
        .groupby(["initDate","date","model","nleadHour","nleadDays"])
        .apply(lambda g: pd.Series({
            "wwind10": (g["wwind10"]*g["weight"]).sum()/g["weight"].sum(),
            "t2m": (g["t2m"]*g["weight"]).sum()/g["weight"].sum(),
            "wwind100": (g["wwind100"]*g["weight"]).sum()/g["weight"].sum(),
            "name": "SEIN"
        }))
        .reset_index())
wind_dataset = pd.concat([wind_dataset, sein], ignore_index=True)

/tmp/ipykernel_17535/3280157497.py:9: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [26]:
dataset

,date,NombreEmpresa,Tipoinfoabrev,power,name
0,2025-09-19 00:30:00,ENGIE ENERGIA PERU S.A.A.,MW,121.3275,W.F. Punta Lomitas
1,2025-09-19 01:00:00,ENGIE ENERGIA PERU S.A.A.,MW,121.3275,W.F. Punta Lomitas
2,2025-09-19 01:30:00,ENGIE ENERGIA PERU S.A.A.,MW,114.4985,W.F. Punta Lomitas
3,2025-09-19 02:00:00,ENGIE ENERGIA PERU S.A.A.,MW,114.4985,W.F. Punta Lomitas
4,2025-09-19 02:30:00,ENGIE ENERGIA PERU S.A.A.,MW,107.2130,W.F. Punta Lomitas
...,...,...,...,...,...
2875,2025-09-24 22:00:00,ORYGEN PERU S.A.A.,MW,56.6980,W.F. Wayra I
2876,2025-09-24 22:30:00,ORYGEN PERU S.A.A.,MW,41.5760,W.F. Wayra I
2877,2025-09-24 23:00:00,ORYGEN PERU S.A.A.,MW,41.5760,W.F. Wayra I
2878,2025-09-24 23:30:00,ORYGEN PERU S.A.A.,MW,27.9155,W.F. Wayra I


In [ ]:
# -*- coding:utf-8 -*-
#-----------------------------------------------------
# @Project: windShortTermForecast
# @File:    app.py
# @Author:  Carlos Enciso Ojeda
#-----------------------------------------------------
import os
import dash
import pandas as pd
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px

# =======================
# Load Generation Dataset
# =======================
WEIGHTS_FARMS = {
    "W.F. Cupisnique": 83.15, "W.F. Duna": 18.37, "W.F. Huambos": 18.37, "W.F. Marcona": 32.0,
    "W.F. Punta Lomitas": 296.4, "W.F. San Juan": 135.7, "W.F. Talara": 30.86,
    "W.F. Tres Hermanas": 97.15, "W.F. Wayra Ext": 177.0, "W.F. Wayra I": 132.30
}

PARQUET_CANDIDATES = [
    os.environ.get("DATASET_PATH"),
    "../dataset/currentGen.parquet",
    "./dataset/currentGen.parquet",
    "/opt/render/project/dataset/currentGen.parquet",
]
parquet_path = next((p for p in PARQUET_CANDIDATES if p and os.path.exists(p)), None)

if parquet_path:
    dataset = pd.read_parquet(parquet_path)
    print(f"[OK] Data loaded from: {parquet_path} | shape={dataset.shape}")
    #-- Compute SEIN generation as weighted average --#
    sein = (dataset.assign(weight=dataset["name"].map(weights))
               .dropna(subset=["weight"])
               .groupby("date")
               .apply(lambda g: pd.Series({
                   "date": g["date"].iloc[0],
                   "NombreEmpresa": "SEIN",
                   "Tipoinfoabrev": "MW",
                   "power": np.average(g["power"], weights=g["weight"]),
                   "name": "SEIN"
               })))
    dataset = pd.concat([dataset, sein.reset_index(drop=True)], ignore_index=True)
else:
    raise FileNotFoundError("currentGen.parquet not found")

# Validate required columns
expected_cols = {"name", "date", "power"}
missing = expected_cols - set(dataset.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")

dataset = dataset.copy()
dataset["date"] = pd.to_datetime(dataset["date"])

# =======================
# Load Wind Forecast Dataset
# =======================
WIND_PARQUET_CANDIDATES = [
    os.environ.get("WIND_FCS_PATH"),
    "../dataset/windSpeedFcs.parquet",
    "./dataset/windSpeedFcs.parquet",
    "/opt/render/project/dataset/windSpeedFcs.parquet",
]
wind_parquet_path = next((p for p in WIND_PARQUET_CANDIDATES if p and os.path.exists(p)), None)

if wind_parquet_path:
    wind_dataset = pd.read_parquet(wind_parquet_path)
    print(f"[OK] Wind data loaded from: {wind_parquet_path} | shape={wind_dataset.shape}")
    
    # Process wind data to keep latest forecast per model
    wind_dataset['initDate'] = pd.to_datetime(wind_dataset['initDate'])
    max_initDate_por_modelo = wind_dataset.groupby('model')['initDate'].transform('max')
    wind_dataset = wind_dataset[wind_dataset['initDate'] == max_initDate_por_modelo].copy()
    
    wind_expected = {"name", "date", "wwind100", "model"}
    wmissing = wind_expected - set(wind_dataset.columns)
    if wmissing:
        for col in wmissing:
            wind_dataset[col] = pd.Series(dtype="float64" if col == "wwind100" else "object")
else:
    print("[WARN] windSpeedFcs.parquet not found; wind chart will be empty")
    wind_dataset = pd.DataFrame(columns=["name", "date", "wwind100", "model"])

wind_dataset = wind_dataset.copy()
wind_dataset["date"] = pd.to_datetime(wind_dataset["date"], errors="coerce")
#-- Adding SEIN name --#


# =======================
# App Configuration
# =======================
app = Dash(
    __name__,
    external_stylesheets=[
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"
    ],
)
app.title = "ENGIE WindForecast Platform"
server = app.server

# Theme settings
LIGHT_THEME = dbc.themes.CERULEAN
DARK_THEME  = dbc.themes.SUPERHERO

# =======================
# Styling Helpers
# =======================
def get_theme(dark: bool):
    colors = {
        "bg":   "#0f2537" if dark else "#ffffff",
        "text": "#ffffff" if dark else "#2c3e50",
    }
    template = "plotly_dark" if dark else "plotly_white"
    rs_style = dict(
        bgcolor="#000000" if dark else "#ffffff",
        activecolor="#4169e1" if dark else "#00bdff",
        font=dict(color=colors["text"]),
    )
    return colors, template, rs_style

def apply_common_layout(fig, *, dark: bool, ytitle: str, x_dtick_ms: int, step='backward'):
    colors, template, rs_style = get_theme(dark)
    fig.update_traces(line=dict(width=2))
    fig.update_layout(
        template=template,
        plot_bgcolor=colors["bg"],
        paper_bgcolor=colors["bg"],
        font=dict(color=colors["text"]),
        xaxis=dict(
            tickfont_size=10,
            showgrid=True, gridcolor="lightgrey", gridwidth=0.01, griddash="solid",
            tickangle=0, tickformat="%d-%b\n%H:%M", dtick=x_dtick_ms, title=""
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey", gridwidth=0.01, griddash="solid",
            title=dict(text=ytitle, font=dict(size=12))
        ),
        hovermode="x unified",
        margin=dict(t=40, b=5, l=10, r=10),
        autosize=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        title=None,
        height=320  # Uniform height for both charts
    )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1D", step="day", stepmode=step),
                dict(count=5, label="5D", step="day", stepmode=step),
                dict(step="all"),
            ],
            **rs_style,
        ),
    )
    return fig

# =======================
# UI Components
# =======================
def create_metric_card(icon_classes: str, title: str, value: str, change, color: str = "primary"):
    try:
        up = (str(change).startswith("+") or float(str(change).replace("+","").replace("%","")) >= 0)
    except Exception:
        up = True
    trend_class = "text-success" if up else "text-danger"
    change_txt  = f"{change}% from last month" if "%" not in str(change) else f"{change} from last month"
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([html.I(className=f"{icon_classes} fa-2x text-{color}")],
                         className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0", style={'font-size': '32px','font-weight': 'bold'}),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(change_txt, className=trend_class)
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")

metric_cards = [
    create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "primary"),
    create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "info"),
    create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
]

color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-sun", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-moon", html_for="switch"),
    ],
    className="my-2"
)

# Windfarm mapping
WINDFARM_MAPPING = {
    "0": "SEIN", "1": "W.F. Punta Lomitas", "2": "W.F. Cupisnique",
    "3": "W.F. Duna", "4": "W.F. Huambos", "5": "W.F. Marcona",
    "6": "W.F. San Juan", "7": "W.F. Talara", "8": "W.F. Tres Hermanas",
    "9": "W.F. Wayra Ext", "10": "W.F. Wayra I"
}

# =======================
# App Layout
# =======================
app.layout = dbc.Container([
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),

    dbc.Row([
        # Left Panel
        dbc.Col([
            html.Div([
                html.Div(color_mode_switch, className='mb-3 text-end'),
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={'width': '200px','margin-bottom': '20px','display': 'block','margin-left': 'auto','margin-right': 'auto'}
                ),
                html.H1([html.Span("Welcome"), html.Br(), html.Span("WindForecast Platform")],
                        style={'color':'#00bdff'}, className="text-center mb-4"),
                html.P(
                    "This platform provides accurate wind forecasts and energy generation "
                    "insights for 10 wind farms and the SEIN system. It supports short and "
                    "long-term planning with interactive tools and clear visualizations.",
                    className="text-justify mb-4",
                ),
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value": 1},
                        {"label": "Long  Term (07 months)", "value": 2},
                    ],
                    value=1,
                ),
                html.Hr(),
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'}),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[{"label": v, "value": k} for k, v in WINDFARM_MAPPING.items()],
                    value="1",
                    className="mb-3"
                ),
            ])
        ], width=4),

        # Right Panel
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'}),
            dbc.Row([dbc.Col(card, width=4) for card in metric_cards]),
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='genPlot', style={'height': '350px'}),
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='windPlot', style={'height': '350px'}),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'}),
        ], width=8)
    ])
],
style={'display':'flex', 'maxWidth':'1440px'},
fluid=True)

# =======================
# Callbacks
# =======================
@app.callback(
    Output("theme-link", "href"),
    Output("theme-store", "data"),
    Input("switch", "value"),
    State("theme-store", "data"),
)
def toggle_theme(switch_value, data):
    dark = bool(switch_value)
    theme_href = DARK_THEME if dark else LIGHT_THEME
    return theme_href, {"dark": dark}

@app.callback(
    Output('genPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_gen_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")
    subset = dataset.query("name == @windfarm_name").copy()
    
    if subset.empty:
        subset = pd.DataFrame({"date": pd.to_datetime([]), "power": []})

    fig = px.line(subset, x='date', y='power', line_shape='linear')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Generation (MW)',
                              x_dtick_ms=18*3600*1000)
    return fig

@app.callback(
    Output('windPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_wind_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    if not wind_dataset.empty and 'name' in wind_dataset.columns:
        subset = wind_dataset.query("name == @windfarm_name").copy()
    else:
        subset = pd.DataFrame(columns=["date", "wwind100", "model"])
        subset["date"] = pd.to_datetime(subset["date"])

    fig = px.line(subset, x='date', y='wwind100', color='model', line_shape='spline')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Speed (m/s)',
                              x_dtick_ms=24*3600*1000, step='todate')
    return fig

# =======================
# Main Execution
# =======================
if __name__ == '__main__':
    app.run(host="127.0.0.1", port=8050, debug=False)

[OK] Data loaded from: ../dataset/currentGen.parquet | shape=(2880, 5)
[OK] Wind data loaded from: ../dataset/windSpeedFcs.parquet | shape=(71270, 9)


In [ ]:
# -*- coding:utf-8 -*-
#-----------------------------------------------------
# @Project: windShortTermForecast
# @File:    app.py
# @Author:  Carlos Enciso Ojeda
#-----------------------------------------------------
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import warnings
warnings.filterwarnings("ignore")

# =======================
# Config de pesos y grupos
# =======================
WEIGHTS_FARMS = {
    "W.F. Cupisnique": 83.15, "W.F. Duna": 18.37, "W.F. Huambos": 18.37, "W.F. Marcona": 32.0,
    "W.F. Punta Lomitas": 296.4, "W.F. San Juan": 135.7, "W.F. Talara": 30.86,
    "W.F. Tres Hermanas": 97.15, "W.F. Wayra Ext": 177.0, "W.F. Wayra I": 132.30
}
GROUPS = {
    "Costa Norte": ["W.F. Cupisnique", "W.F. Talara"],
    "Costa Sur": ["W.F. Punta Lomitas", "W.F. Marcona", "W.F. Tres Hermanas", "W.F. Wayra Ext", "W.F. Wayra I"],
    "Sierra Centro": ["W.F. Huambos", "W.F. Duna", "W.F. San Juan"],
}

# =======================
# Load Generation Dataset
# =======================
PARQUET_CANDIDATES = [
    os.environ.get("DATASET_PATH"),
    "../dataset/currentGen.parquet",
    "./dataset/currentGen.parquet",
    "/opt/render/project/dataset/currentGen.parquet",
]
parquet_path = next((p for p in PARQUET_CANDIDATES if p and os.path.exists(p)), None)
if not parquet_path:
    raise FileNotFoundError("currentGen.parquet not found")

dataset = pd.read_parquet(parquet_path)
print(f"[OK] Data loaded from: {parquet_path} | shape={dataset.shape}")

# Validaciones mínimas
expected_cols = {"name", "date", "power"}
missing = expected_cols - set(dataset.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")
dataset = dataset.copy()
dataset["date"] = pd.to_datetime(dataset["date"])

# =======================
# Load Wind Forecast Dataset
# =======================
WIND_PARQUET_CANDIDATES = [
    os.environ.get("WIND_FCS_PATH"),
    "../dataset/windSpeedFcs.parquet",
    "./dataset/windSpeedFcs.parquet",
    "/opt/render/project/dataset/windSpeedFcs.parquet",
]
wind_parquet_path = next((p for p in WIND_PARQUET_CANDIDATES if p and os.path.exists(p)), None)

if wind_parquet_path:
    wind_dataset = pd.read_parquet(wind_parquet_path)
    print(f"[OK] Wind data loaded from: {wind_parquet_path} | shape={wind_dataset.shape}")
    # Mantener última corrida por modelo
    if "initDate" in wind_dataset.columns:
        wind_dataset["initDate"] = pd.to_datetime(wind_dataset["initDate"], errors="coerce")
        max_initDate_por_modelo = wind_dataset.groupby('model')['initDate'].transform('max')
        wind_dataset = wind_dataset[wind_dataset['initDate'] == max_initDate_por_modelo].copy()
    # Columnas mínimas
    wind_expected = {"name", "date", "wwind100", "model"}
    for col in wind_expected - set(wind_dataset.columns):
        wind_dataset[col] = pd.Series(dtype="float64" if col == "wwind100" else "object")
else:
    print("[WARN] windSpeedFcs.parquet not found; wind chart will be empty")
    wind_dataset = pd.DataFrame(columns=["name", "date", "wwind100", "model"])

wind_dataset = wind_dataset.copy()
wind_dataset["date"] = pd.to_datetime(wind_dataset["date"], errors="coerce")

# =======================
# App Configuration
# =======================
app = Dash(
    __name__,
    external_stylesheets=[
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"
    ],
)
app.title = "ENGIE WindForecast Platform"
server = app.server

# Theme settings
LIGHT_THEME = dbc.themes.CERULEAN
DARK_THEME  = dbc.themes.SUPERHERO

# =======================
# Styling Helpers
# =======================
def get_theme(dark: bool):
    colors = {
        "bg":   "#0f2537" if dark else "#ffffff",
        "text": "#ffffff" if dark else "#2c3e50",
    }
    template = "plotly_dark" if dark else "plotly_white"
    rs_style = dict(
        bgcolor="#000000" if dark else "#ffffff",
        activecolor="#4169e1" if dark else "#00bdff",
        font=dict(color=colors["text"]),
    )
    return colors, template, rs_style

def apply_common_layout(fig, *, dark: bool, ytitle: str, x_dtick_ms: int, step='backward'):
    colors, template, rs_style = get_theme(dark)
    fig.update_traces(line=dict(width=2))
    fig.update_layout(
        template=template,
        plot_bgcolor=colors["bg"],
        paper_bgcolor=colors["bg"],
        font=dict(color=colors["text"]),
        xaxis=dict(
            tickfont_size=10,
            showgrid=True, gridcolor="lightgrey", gridwidth=0.5,
            tickangle=0, tickformat="%d-%b\n%H:%M", dtick=x_dtick_ms, title=""
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey", gridwidth=0.5,
            title=dict(text=ytitle, font=dict(size=12))
        ),
        hovermode="x unified",
        margin=dict(t=30, b=20, l=10, r=10),
        autosize=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.00, xanchor="right", x=1),
        title=None,
        height=320
    )
    fig.update_xaxes(
        rangeslider_visible=False,
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1D", step="day", stepmode=step),
                dict(count=5, label="5D", step="day", stepmode=step),
                dict(step="all"),
            ],
            **rs_style,
        ),
    )
    return fig

# =======================
# UI Components
# =======================
def create_metric_card(icon_classes: str, title: str, value: str, change, color: str = "primary"):
    try:
        up = (str(change).startswith("+") or float(str(change).replace("+","").replace("%","")) >= 0)
    except Exception:
        up = True
    trend_class = "text-success" if up else "text-danger"
    change_txt  = f"{change}% from last month" if "%" not in str(change) else f"{change} from last month"
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([html.I(className=f"{icon_classes} fa-2x text-{color}")],
                         className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0", style={'font-size': '32px','font-weight': 'bold'}),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(change_txt, className=trend_class)
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")

metric_cards = [
    create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "primary"),
    create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "info"),
    create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
]

color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-sun", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-moon", html_for="switch"),
    ],
    className="my-2"
)

# Windfarm mapping
WINDFARM_MAPPING = {
    "0": "SEIN", "1": "W.F. Punta Lomitas", "2": "W.F. Cupisnique",
    "3": "W.F. Duna", "4": "W.F. Huambos", "5": "W.F. Marcona",
    "6": "W.F. San Juan", "7": "W.F. Talara", "8": "W.F. Tres Hermanas",
    "9": "W.F. Wayra Ext", "10": "W.F. Wayra I"
}

# =======================
# Helpers para series ponderadas por grupos
# =======================
def _weighted_series(df, names, value_col, weights):
    tmp = (df[df["name"].isin(names)]
           .assign(w=lambda x: x["name"].map(weights))
           .dropna(subset=["w"]))
    if tmp.empty:
        return pd.Series(dtype="float64")
    s = tmp.groupby("date").apply(lambda g: np.average(g[value_col], weights=g["w"]))
    s.index = pd.to_datetime(s.index)
    return s.sort_index()

def _build_group_figure(df, value_col, ytitle, dark, weights, x_dtick_ms):
    traces = []
    for gname, names in GROUPS.items():
        s = _weighted_series(df, names, value_col=value_col, weights=weights)
        traces.append(go.Scatter(x=s.index, y=s.values, mode="lines", name=gname))
    # Ponderado nacional (todas las granjas)
    all_names = list(weights.keys())
    s_all = _weighted_series(df, all_names, value_col=value_col, weights=weights)
    traces.append(go.Scatter(x=s_all.index, y=s_all.values, mode="lines",
                             name="SEIN (Total)", line=dict(width=3)))
    fig = go.Figure(data=traces)
    fig = apply_common_layout(fig, dark=bool(dark), ytitle=ytitle, x_dtick_ms=x_dtick_ms)
    return fig

# =======================
# App Layout
# =======================
app.layout = dbc.Container([
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),

    dbc.Row([
        # Left Panel
        dbc.Col([
            html.Div([
                html.Div(color_mode_switch, className='mb-3 text-end'),
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={'width': '200px','margin-bottom': '20px','display': 'block','margin-left': 'auto','margin-right': 'auto'}
                ),
                html.H1([html.Span("Welcome"), html.Br(), html.Span("WindForecast Platform")],
                        style={'color':'#00bdff'}, className="text-center mb-4"),
                html.P(
                    "This platform provides accurate wind forecasts and energy generation "
                    "insights for 10 wind farms and the SEIN system. It supports short and "
                    "long-term planning with interactive tools and clear visualizations.",
                    className="text-justify mb-4",
                ),
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value": 1},
                        {"label": "Long  Term (07 months)", "value": 2},
                    ],
                    value=1,
                ),
                html.Hr(),
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'}),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[{"label": v, "value": k} for k, v in WINDFARM_MAPPING.items()],
                    value="1",
                    className="mb-3"
                ),
            ])
        ], width=4),

        # Right Panel
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'}),
            dbc.Row([dbc.Col(card, width=4) for card in metric_cards]),
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='genPlot', style={'height': '350px'}),
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='windPlot', style={'height': '350px'}),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'}),
        ], width=8)
    ])
],
style={'display':'flex', 'maxWidth':'1440px'},
fluid=True)

# =======================
# Callbacks
# =======================
@app.callback(
    Output("theme-link", "href"),
    Output("theme-store", "data"),
    Input("switch", "value"),
    State("theme-store", "data"),
)
def toggle_theme(switch_value, data):
    dark = bool(switch_value)
    theme_href = DARK_THEME if dark else LIGHT_THEME
    return theme_href, {"dark": dark}

@app.callback(
    Output('genPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_gen_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    if windfarm_name == "SEIN":
        # 3 zonas + SEIN ponderado
        return _build_group_figure(dataset, value_col="power",
                                   ytitle="Wind Generation (MW)",
                                   dark=bool(dark), weights=WEIGHTS_FARMS,
                                   x_dtick_ms=18*3600*1000)

    subset = dataset.query("name == @windfarm_name").copy()
    if subset.empty:
        subset = pd.DataFrame({"date": pd.to_datetime([]), "power": []})

    fig = px.line(subset, x='date', y='power', line_shape='linear')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Generation (MW)',
                              x_dtick_ms=18*3600*1000)
    return fig

@app.callback(
    Output('windPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_wind_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    if windfarm_name == "SEIN":
        # Agregar por zonas ponderadas y promedio entre modelos
        def _weighted_series_wind(df, names, weights):
            tmp = (df[df["name"].isin(names)]
                   .assign(w=lambda x: x["name"].map(weights))
                   .dropna(subset=["w"]))
            if tmp.empty:
                return pd.Series(dtype="float64")
            # Pondero por granja dentro de cada modelo y fecha
            per_model = tmp.groupby(["date", "model"]).apply(
                lambda g: np.average(g["wwind100"], weights=g["w"])
            ).reset_index(name="v")
            # Promedio entre modelos por fecha
            s = per_model.groupby("date")["v"].mean()
            s.index = pd.to_datetime(s.index)
            return s.sort_index()

        traces = []
        for gname, names in GROUPS.items():
            s = _weighted_series_wind(wind_dataset, names, WEIGHTS_FARMS)
            traces.append(go.Scatter(x=s.index, y=s.values, mode="lines", name=gname))

        all_names = list(WEIGHTS_FARMS.keys())
        s_all = _weighted_series_wind(wind_dataset, all_names, WEIGHTS_FARMS)
        traces.append(go.Scatter(x=s_all.index, y=s_all.values, mode="lines",
                                 name="SEIN (Total)", line=dict(width=3)))

        fig = go.Figure(traces)
        fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Speed (m/s)',
                                  x_dtick_ms=24*3600*1000, step='todate')
        return fig

    # Caso normal: por parque, coloreado por modelo
    if not wind_dataset.empty and 'name' in wind_dataset.columns:
        subset = wind_dataset.query("name == @windfarm_name").copy()
    else:
        subset = pd.DataFrame(columns=["date", "wwind100", "model"])
        subset["date"] = pd.to_datetime(subset["date"])

    fig = px.line(subset, x='date', y='wwind100', color='model', line_shape='spline')
    fig = apply_common_layout(fig, dark=bool(dark), ytitle='Wind Speed (m/s)',
                              x_dtick_ms=24*3600*1000, step='todate')
    return fig

# =======================
# Main Execution
# =======================
if __name__ == '__main__':
    app.run(host="127.0.0.1", port=8050, debug=False)


[OK] Data loaded from: ../dataset/currentGen.parquet | shape=(2880, 5)
[OK] Wind data loaded from: ../dataset/windSpeedFcs.parquet | shape=(71270, 9)


[2025-09-24 16:47:00,680] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/cenciso/anaconda3/envs/dash/lib/python3.10/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/cenciso/anaconda3/envs/dash/lib/python3.10/site-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/cenciso/anaconda3/envs/dash/lib/python3.10/site-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/cenciso/anaconda3/envs/dash/lib/python3.10/site-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "/home/cenciso/anaconda3/envs/dash/lib/python3.10/site-packages/dash/dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "/home/cenciso/anaconda3/envs/da

In [18]:
# -*- coding:utf-8 -*-
#-----------------------------------------------------
# @Project: ./windShortTermForecast/app/
# @File:    ./windShortTermForecast/app/app.py
# @Author:  Carlos Enciso Ojeda
#-----------------------------------------------------
# -- Imports -- #
import os
import dash
import pandas as pd
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px

# =======================
# Read Gen Dataset (Parquet)
# =======================
# Prioridad de rutas: ENV -> local repo -> Render path (por si reusas)
PARQUET_CANDIDATES = [
    os.environ.get("DATASET_PATH"),
    "../dataset/currentGen.parquet",
    "./dataset/currentGen.parquet",
    "/opt/render/project/dataset/currentGen.parquet",
]
parquet_path = next((p for p in PARQUET_CANDIDATES if p and os.path.exists(p)), None)

if parquet_path:
    dataset = pd.read_parquet(parquet_path)
    print(f"[OK] Data loaded from: {parquet_path} | shape={dataset.shape}")
else:
    # Falla segura con mensaje claro
    raise FileNotFoundError(
        "No se encontró 'currentGen.parquet'. "
        "Colócalo en ./dataset/ o ../dataset/ o define DATASET_PATH."
    )

# Validación mínima de columnas esperadas
expected_cols = {"name", "date", "power"}
missing = expected_cols - set(dataset.columns)
if missing:
    raise ValueError(f"El Parquet no contiene columnas requeridas {expected_cols}. "
                     f"Faltan: {missing}")

# Asegurar tipos
dataset = dataset.copy()
dataset["date"] = pd.to_datetime(dataset["date"])

# =======================
# App & Theme Setup
# =======================
app = Dash(
    __name__,
    external_stylesheets=[
        # No cargar tema dbc aquí; lo alternamos con <link> dinámico
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"
    ],
)
app.title = "ENGIE WindForecast Platform"
server = app.server  # por si luego lo usas en un WSGI

# Temas (light/dark)
LIGHT_THEME = dbc.themes.CERULEAN
DARK_THEME  = dbc.themes.SUPERHERO

# =======================
# UI Bits
# =======================
def create_metric_card(icon, title, value, change, color="primary"):
    up = (str(change).startswith("+") or float(str(change).replace("+","").replace("%","")) >= 0)
    trend_class = "text-success" if up else "text-danger"
    change_txt  = f"{change}% from last month" if "%" not in str(change) else f"{change} from last month"
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([html.I(className=f"fas {icon} fa-2x text-{color}")],
                         className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0", style={'font-size': '32px','font-weight': 'bold'}),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(change_txt, className=trend_class)
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")

metric_cards = [
    create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "primary"),
    create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "info"),
    create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
]

color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-sun", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-moon", html_for="switch"),
    ],
    className="my-2"
)

# Mapeo de valores del RadioItem -> nombres en dataset['name']
WINDFARM_MAPPING = {
    "0": "SEIN", "1": "W.F. Punta Lomitas", "2": "W.F. Cupisnique",
    "3": "W.F. Duna", "4": "W.F. Huambos", "5": "W.F. Marcona",
    "6": "W.F. San Juan", "7": "W.F. Talara", "8": "W.F. Tres Hermanas",
    "9": "W.F. Wayra Ext", "10": "W.F. Wayra I"
}

# =======================
# Layout
# =======================
app.layout = dbc.Container([
    # Theme link y estado (para toggle dinámico)
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),

    dbc.Row([
        # -------- Left Panel -------- #
        dbc.Col([
            html.Div([
                html.Div(color_mode_switch, className='mb-3 text-end'),
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={'width': '200px','margin-bottom': '20px','display': 'block','margin-left': 'auto','margin-right': 'auto'}
                ),
                html.H1([html.Span("Welcome"), html.Br(), html.Span("WindForecast Platform")],
                        style={'color':'#00bdff'}, className="text-center mb-4"),
                html.P(
                    "This platform provides accurate wind forecasts and energy generation "
                    "insights for 10 wind farms and the SEIN system. It supports short and "
                    "long-term planning with interactive tools and clear visualizations.",
                    className="text-justify mb-4",
                ),
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value": 1},
                        {"label": "Long  Term (07 months)", "value": 2},
                    ],
                    value=1,
                ),
                html.Hr(),
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'}),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[
                        {"label": "SEIN", "value": "0"},
                        {"label": "W.F. Punta Lomitas", "value": "1"},
                        {"label": "W.F. Cupisnique", "value": "2"},
                        {"label": "W.F. Duna", "value": "3"},
                        {"label": "W.F. Huambos", "value": "4"},
                        {"label": "W.F. Marcona", "value": "5"},
                        {"label": "W.F. San Juan", "value": "6"},
                        {"label": "W.F. Talara", "value": "7"},
                        {"label": "W.F. Tres Hermanas", "value": "8"},
                        {"label": "W.F. Wayra Ext", "value": "9"},
                        {"label": "W.F. Wayra I", "value": "10"},
                    ],
                    value="1",
                    className="mb-3"
                ),
            ])
        ], width=4),

        # -------- Right Panel -------- #
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'}),
            dbc.Row([dbc.Col(card, width=4) for card in metric_cards]),
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='genPlot', style={'height': '400px'}),
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'}),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'}),
        ], width=8)
    ])
],
style={'display':'flex', 'maxWidth':'1440px'},
fluid=True)

# =======================
# Callbacks
# =======================
@app.callback(
    Output("theme-link", "href"),
    Output("theme-store", "data"),
    Input("switch", "value"),
    State("theme-store", "data"),
)
def toggle_theme(switch_value, data):
    dark = bool(switch_value)
    theme_href = DARK_THEME if dark else LIGHT_THEME
    return theme_href, {"dark": dark}

@app.callback(
    Output('genPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def update_gen_plot(windfarm_value, dark):
    windfarm_name = WINDFARM_MAPPING.get(windfarm_value, "SEIN")

    # Subset robusto (si no hay datos, crear DF vacío para evitar crash)
    subset = dataset.query("name == @windfarm_name").copy()
    if subset.empty:
        subset = pd.DataFrame({"date": [], "power": []})

    fig = px.line(subset, x='date', y='power', line_shape='linear')

    # Estilo base por tema
    fig.update_traces(line=dict(width=1.2))
    fig.update_layout(
        template="plotly_dark" if dark else "plotly_white",
        xaxis=dict(
            tickfont_size=10,
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=0.01,
            tickangle=0,
            tickformat="%d-%b\n%H:%M hrs.",
            dtick=18*3600000,  # cada 18 horas
            griddash='solid',
            title=''
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
            gridwidth=0.01,
            griddash='solid',
            title=dict(text='Wind Generation (MW)', font=dict(size=12))
        ),
        hovermode="x unified",
        margin=dict(t=10, b=10, l=10, r=10),
        autosize=True,
        title=None
    )

    # Range slider + rangeselector con colores acordes al tema
    rs_style = dict(
        bgcolor="#000000" if dark else "#ffffff",
        activecolor="#4169e1" if dark else "#00bdff",
        font=dict(color="#ffffff" if dark else "#000000"),
    )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1D", step="day", stepmode="backward"),
                dict(count=5, label="5D", step="day", stepmode="backward"),
                dict(step="all")
            ],
            **rs_style
        )
    )
    return fig

# =======================
# Main
# =======================
if __name__ == '__main__':
    # Local run
    app.run(host="127.0.0.1", port=8050, debug=False)


[OK] Data loaded from: ../dataset/currentGen.parquet | shape=(2880, 5)


In [2]:
import dash
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px

#-- Layout --#
app = Dash(__name__,
           external_stylesheets=[dbc.themes.CERULEAN])

#-- Theme to Toggle --#
LIGHT_THEME = dbc.themes.CERULEAN
DARK_THEME = dbc.themes.SUPERHERO

#-- Function to create sample figures --#
def make_figures(dark=False):
    # Sample data for demonstration
    import pandas as pd
    import numpy as np

    # Generate sample wind forecast data
    dates = pd.date_range('2024-01-01', periods=24, freq='H')
    wind_speed = np.random.uniform(5, 25, 24)
    wind_direction = np.random.uniform(0, 360, 24)

    # Color scheme based on theme
    colors = {
        'background': '#2c3e50' if dark else '#ffffff',
        'text': '#ffffff' if dark else '#2c3e50',
        'primary': '#3498db'
    }

    # Create scatter plot
    fig_scatter = go.Figure()
    fig_scatter.add_trace(go.Scatter(
        x=dates,
        y=wind_speed,
        mode='lines+markers',
        name='Wind Speed (m/s)',
        line=dict(color=colors['primary'])
    ))

    fig_scatter.update_layout(
        title='Wind Speed Forecast',
        xaxis_title='Time',
        yaxis_title='Wind Speed (m/s)',
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font=dict(color=colors['text']),
        height=350
    )

    # Create bar chart
    fig_bar = go.Figure()
    fig_bar.add_trace(go.Bar(
        x=dates[:12],
        y=wind_speed[:12],
        name='Hourly Wind Speed',
        marker_color=colors['primary']
    ))

    fig_bar.update_layout(
        title='Wind Speed Distribution',
        xaxis_title='Time',
        yaxis_title='Wind Speed (m/s)',
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font=dict(color=colors['text']),
        height=350
    )

    return fig_scatter, fig_bar

#-- UI: Theme switch --#
color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-moon", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-sun", html_for="switch"),
    ],
    className="switch"
)

#-- Main Layout --#
app.layout = dbc.Container([
    #-- Theme link and store --#
    dbc.Label(className="fa fa-moon", html_for="switch"),
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),

    dbc.Row([
        #-- Left Panel --#
        dbc.Col([
            html.Div([
                # Theme switch
                html.Div(color_mode_switch, className="mb-3 text-end"),

                # Logo
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={
                        'width': '200px',
                        'margin-bottom': '20px',
                        'display': 'block',
                        'margin-left': 'auto',
                        'margin-right': 'auto'
                    }
                ),

                # Title
                html.H1([
                    html.Span("Welcome"),
                    html.Br(),
                    html.Span("WindForecast Platform")
                ], className="text-center mb-4"),

                # Description
                html.P(
                    "This platform was created for wind forecast in an effective way",
                    className="text-center mb-4"
                ),

                # Time range buttons
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-vertical',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (24h)", "value": 1},
                        {"label": "Long Term (7 days)", "value": 2},
                    ],
                    value=1,
                ),

                html.Hr(),

                # Wind farms section
                html.H5("WindFarms", className="mb-3"),
                dbc.Checklist(
                    id="windfarm-checklist",
                    options=[
                        {"label": "Farm Alpha", "value": "alpha"},
                        {"label": "Farm Beta", "value": "beta"},
                        {"label": "Farm Gamma", "value": "gamma"},
                    ],
                    value=["alpha"],
                    className="mb-3"
                ),
            ])
        ], width=4),

        #-- Right Panel --#
        dbc.Col([
            html.H3("Wind Forecast Dashboard", className="mb-4"),

            # Graphs
            dbc.Row([
                dbc.Col([
                    dcc.Graph(id="scatter-graph")
                ], width=12)
            ]),

            dbc.Row([
                dbc.Col([
                    dcc.Graph(id="bar-graph")
                ], width=12)
            ]),

        ], width=8)
    ])
], fluid=True)

#-- Callbacks --#
@app.callback(
    Output("theme-link", "href"),
    Output("scatter-graph", "figure"),
    Output("bar-graph", "figure"),
    Output("theme-store", "data"),
    Input("switch", "value"),
    State("theme-store", "data"),
)
def toggle_theme(switch_value, data):
    dark = bool(switch_value)
    theme_href = DARK_THEME if dark else LIGHT_THEME
    fig_scatter, fig_bar = make_figures(dark=dark)
    return theme_href, fig_scatter, fig_bar, {"dark": dark}

@app.callback(
    Output("scatter-graph", "figure", allow_duplicate=True),
    Output("bar-graph", "figure", allow_duplicate=True),
    Input("top-buttons", "value"),
    Input("windfarm-checklist", "value"),
    State("theme-store", "data"),
    prevent_initial_call=True
)
def update_graphs(time_range, selected_farms, theme_data):
    dark = theme_data.get("dark", False)
    # Here you would filter data based on time_range and selected_farms
    fig_scatter, fig_bar = make_figures(dark=dark)
    return fig_scatter, fig_bar

if __name__ == '__main__':
    app.run(debug=True, port=8049)

/tmp/ipykernel_346786/2050309801.py:22: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [1]:
# -*- coding:utf-8 -*-
#-----------------------------------------------------
# @Project: ./windShortTermForecast/app/
# @File: ./windShortTermForecast/app/app.py
# @Author: Carlos Enciso Ojeda
# @Email: carlos.enciso.o@gmail.com
#-----------------------------------------------------
#-- Import modules --#
import dash
import time
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

#-- Function to generate sample data for demonstration --#
def generate_sample_data(windfarm_id):
    """Generate sample wind generation data for different wind farms"""
    np.random.seed(int(windfarm_id))  # For consistent results
    
    # Create date range for the next 15 days
    dates = [datetime.now() + timedelta(hours=i) for i in range(24*15)]
    
    # Different generation patterns based on windfarm
    if windfarm_id == "0":  # SEIN
        base_power = np.random.uniform(800, 1200, len(dates))
        daily_pattern = np.sin(np.linspace(0, 4*np.pi, len(dates))) * 200
        noise = np.random.normal(0, 50, len(dates))
        generation = base_power + daily_pattern + noise
        farm_name = "SEIN System"
    else:
        # Individual wind farms have different patterns
        base_multiplier = int(windfarm_id) * 50
        base_power = np.random.uniform(100 + base_multiplier, 200 + base_multiplier, len(dates))
        daily_pattern = np.sin(np.linspace(0, 4*np.pi, len(dates))) * (50 + int(windfarm_id)*5)
        noise = np.random.normal(0, 10 + int(windfarm_id), len(dates))
        generation = base_power + daily_pattern + noise
        farm_name = f"W.F. {['Punta Lomitas', 'Cupisnique', 'Duna', 'Huambos', 'Marcona', 'San Juan', 'Talara', 'Tres Hermanas', 'Wayra Ext.', 'Wayra I'][int(windfarm_id)-1]}"
    
    return pd.DataFrame({
        'datetime': dates,
        'generation_mw': generation,
        'windfarm': farm_name
    })

#-- Layout --#
app = Dash(__name__,
           external_stylesheets=[
               dbc.themes.SUPERHERO,
               "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"],
           assets_folder='/content/assets',
           )
#-- Tab Name --#
app.title = "ENGIE WindForecast Platform "

#-- Theme to Toggle --#
LIGHT_THEME = dbc.themes.SUPERHERO
DARK_THEME = dbc.themes.SUPERHERO

#-- Cards --#
def create_metric_card(icon, title, value, change, color="primary"):
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([
                    html.I(className=f"fas {icon} fa-2x text-{color}")
                ], className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0",
                            style={
                                'font-size': '32px',
                                'font-weight': 'bold',
                                }),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(f"{change}% from last month",
                              className=f"text-{'success' if '+' in change else 'danger'}")
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")

metric_cards = [
    create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "primary"),
    create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "info"),
    create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
]

#-- UI Theme switch --#
color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-moon", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-sun", html_for="switch"),
    ],
    className="my-2"
)

#-- Main Layout --#
app.layout = dbc.Container([
    #-- Theme link and store --#
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),
    
    #-- Left Panel --#
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Div(color_mode_switch, className='mb-3 text-end'),
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={
                        'width': '200px',
                        'margin-bottom': '20px',
                        'display': 'block',
                        'margin-left': 'auto',
                        'margin-right': 'auto'
                    }
                ),
                html.H1([
                    html.Span("Welcome"),
                    html.Br(),
                    html.Span("WindForecast Platform")
                ], style={'color':'#00bdff'}, className="text-center mb-4"),
                html.P(
                    "This platform provides accurate wind forecasts and energy generation \
                    insights for 10 wind farms and the SEIN system. It supports short and \
                    long-term planning with interactive tools and clear visualizations.",
                    className="text-justify mb-4",
                ),
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                                        labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value":1},
                        {"label": "Long  Term (07 months)", "value":2},
                    ],
                    value=1,
                ),
                html.Hr(),
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'}),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[
                        {"label": "SEIN", "value": "0"},
                        {"label": "W.F. Punta Lomitas", "value": "1"},
                        {"label": "W.F. Cupisnique", "value": "2"},
                        {"label": "W.F. Duna", "value": "3"},
                        {"label": "W.F. Huambos", "value": "4"},
                        {"label": "W.F. Marcona", "value": "5"},
                        {"label": "W.F. San Juan", "value": "6"},
                        {"label": "W.F. Talara", "value": "7"},
                        {"label": "W.F. Tres Hermanas", "value": "8"},
                        {"label": "W.F. Wayra Ext.", "value": "9"},
                        {"label": "W.F. Wayra I", "value": "10"},
                    ],
                    value="0",
                    className="mb-3"
                ),
            ])
        ], width=4),
        
        #-- Right Panel --#
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'}),
            dbc.Row([dbc.Col(card, width=4) for card in metric_cards]),
            
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'}),
            #-- Add the generation plot here --#
            dcc.Graph(id='generation-plot', style={'height': '400px'}),
            
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'}),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'}),
        ], width=8)
    ])
], fluid=True)

#-- Callback to update the generation plot based on windfarm selection --#
@app.callback(
    Output('generation-plot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input('top-buttons', 'value')
)
def update_generation_plot(windfarm_id, forecast_range):
    # Generate sample data based on selected windfarm
    df = generate_sample_data(windfarm_id)
    
    # Create the plot using Plotly Express
    fig = px.line(
        df, 
        x='datetime', 
        y='generation_mw',
        title=f'Wind Generation Forecast - {df["windfarm"].iloc[0]}',
        labels={'datetime': 'Date', 'generation_mw': 'Generation (MW)'}
    )
    
    # Customize the plot appearance
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font_color='white',
        xaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray'),
        yaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray'),
        hovermode='x unified'
    )
    
    # Add a range slider for better navigation
    fig.update_xaxes(rangeslider_visible=True)
    
    return fig

if __name__ == '__main__':
    app.run(mode='inline', debug=False, port=8085)

Address already in use
Port 8085 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'